In [22]:
"""
Collaborative Filtering ALS Recommender System using Spark MLlib 
"""

import os
import numpy as np
import pandas as pd
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from pyspark.mllib.recommendation import ALS
from pyspark.ml.recommendation import ALS as mlals
from pyspark.ml.evaluation import RegressionEvaluator

import math
from pyspark.sql.functions import udf
from pyspark.sql.types import *
from pyspark.ml.evaluation import RegressionEvaluator

In [23]:
# Calling spark session to register application
spark = SparkSession \
    .builder \
    .appName("Recom") \
    .config("spark.recom.demo", "1") \
    .getOrCreate()

In [24]:
"""
Loading and Parsing Dataset
    Each line in the ratings dataset (ratings.csv) is formatted as:
         userId,movieId,rating
""" 

# Load ratings
ratings_df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load("ratings-5.csv")

In [25]:
ratings_df

DataFrame[userId: int, movieId: int, rating: double]

In [26]:
"""
Displaying the first 30 rows
"""
ratings_df.show(30)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|   149|      2|   1.0|
|   149|      3|   0.5|
|   149|      5|   0.5|
|   149|      7|  0.25|
|   149|      9|  0.75|
|   149|     10|  0.75|
|   149|     11|  0.75|
|   149|     12|  0.75|
|   149|     13|  0.75|
|   149|     14|   0.5|
|   149|     16|   1.0|
|   149|     18|  0.75|
|   149|     20|   1.0|
|   149|     21|  0.75|
|   969|      5|   1.0|
|   969|     14|   0.5|
|   969|     15|   0.0|
|   969|     16|   0.0|
|   969|     21|  0.25|
|   589|      1|  0.75|
|   589|      2|   1.0|
|   589|      3|   1.0|
|   589|      4|  0.75|
|   589|      5|   0.5|
|   589|      6|   0.5|
|   589|      7|  0.25|
|   589|      8|  0.75|
|   589|      9|   1.0|
|   589|     10|   1.0|
|   589|     11|   0.5|
+------+-------+------+
only showing top 30 rows



In [27]:
"""
We need first to split it into train, validation, and test datasets.
"""
(trainingData,validationData,testData) = ratings_df.randomSplit([0.6,0.2,0.2])

In [28]:
# Prepare test and validation set. They should not have ratings

validation_for_predict = validationData.select('userId','movieId')
test_for_predict = testData.select('userId','movieId')

In [29]:
"""
Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by 
using Alternating Least Squares. The implementation in MLlib has the following parameters:

    1. numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
    2. rank is the number of latent factors in the model.
    3. iterations is the number of iterations to run.
    4. lambda specifies the regularization parameter in ALS.
    5. implicitPrefs specifies whether to use the explicit 
        feedback ALS variant or one adapted for implicit feedback data.
    6. alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline 
        confidence in preference observations.

"""

seed = 5 #Random seed for initial matrix factorization model. A value of None will use system time as the seed.
iterations = 10
regularization_parameter = 0.1 #run for different lambdas - e.g. 0.01
ranks = [4, 8, 12, 16, 20, 24, 28, 32, 36, 40, 44, 48, 52, 56, 60 ] #number of features
errors = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ]
err = 0
tolerance = 0.02

min_error = float('inf')
best_rank = -1
best_iteration = -1

In [30]:
# Let us traing our dataset and check the best rank with lowest RMSE
# predictAll method of the ALS takes only RDD format and hence we need to convert our dataframe into RDD
# df.rdd will automatically converts Dataframe into RDD

for rank in ranks:
    model = ALS.train(trainingData, rank, seed=seed, iterations=iterations,
                      lambda_=regularization_parameter)
    predictions = model.predictAll(validation_for_predict.rdd).map(lambda r: ((r[0], r[1]), r[2]))
    rates_and_preds = validationData.rdd.map(lambda r: ((int(r[0]), int(r[1])), float(r[2]))).join(predictions)
    error = math.sqrt(rates_and_preds.map(lambda r: (r[1][0] - r[1][1])**2).mean()) # RMSE Error
    errors[err] = error
    err += 1
    print ('For rank %s the RMSE is %s' % (rank, error))
    if error < min_error:
        min_error = error
        best_rank = rank

print ('The best model was trained with rank %s' % best_rank)

For rank 4 the RMSE is 0.2652827409193689
For rank 8 the RMSE is 0.2658825829231328
For rank 12 the RMSE is 0.2658811924571415
For rank 16 the RMSE is 0.2658651029976035
For rank 20 the RMSE is 0.26593694808014695
For rank 24 the RMSE is 0.2657951938394082
For rank 28 the RMSE is 0.26579954298981134
For rank 32 the RMSE is 0.2657838882771415
For rank 36 the RMSE is 0.2658659055147169
For rank 40 the RMSE is 0.2658527589341894
For rank 44 the RMSE is 0.26585588773318863
For rank 48 the RMSE is 0.2658212646542218
For rank 52 the RMSE is 0.2657898387562681
For rank 56 the RMSE is 0.26579961236562233
For rank 60 the RMSE is 0.2658507677322246
The best model was trained with rank 4


In [31]:
"""
ML packages with standard machine learning implementation
Using Regression Evaluator to compute RMSE
The library will take care of RDD itself
"""
als =  mlals(maxIter=iterations,rank=4,seed=seed,regParam=regularization_parameter, userCol="userId", itemCol="movieId",ratingCol="rating")
modelML = als.fit(trainingData)
pred = modelML.transform(validationData)
pred = pred.where(pred['prediction'] != 'NaN')
    
# Evaluate the model by computing RMSE
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(pred)

print ('RMSE is %s' % rmse)

RMSE is 0.26525689239773637


In [32]:
# Let's take test dataset and get ratings
predictions_test = model.predictAll(test_for_predict.rdd).map(lambda r: ((r[0], r[1]), r[2]))

In [33]:
## visualize preditions, here third element is predictions generated by ALS Model
predictions_test.take(5)

[((200, 12), 0.5694241685728765),
 ((200, 10), 0.5160147446315718),
 ((460, 8), 0.71017244290452),
 ((460, 5), 0.7238083670196206),
 ((240, 8), 0.6189074105257013)]

In [34]:
"""
getRecommendations to get recommendations for a particular user from test data

TODO: You need to execute one more step before calling getRecommendations, 
      Think about that step. If you go through the seps below, you will realize it soon.
"""
def getRecommendations(user,testDf,trainDf,model):
    # get all user and his/her rated movies
    userDf = testDf.filter(testDf.userId == user)
    # filter movies from main set which have not been rated by selected user
    # and pass it to model we sreated above
    mov = trainDf.select('movieId').subtract(userDf.select('movieId'))
    
    # Again we need to covert our dataframe into RDD
    pred_rat = model.predictAll(mov.rdd.map(lambda x: (user, x[0]))).collect()
    
    # Get the top recommendations
    recommendations = sorted(pred_rat, key=lambda x: x[2], reverse=True)[:50]
    
    return recommendations

In [35]:
# Assign user id for which we need recommendations
user = 149

# Call getRecommendations method
derived_rec = getRecommendations(user,testData,trainingData,model)

print ("Movies recommended for:%d" % user)



Movies recommended for:149


In [36]:
derived_rec


[Rating(user=149, product=16, rating=0.6447600364992911),
 Rating(user=149, product=13, rating=0.6215800764200071),
 Rating(user=149, product=3, rating=0.6177792189028436),
 Rating(user=149, product=18, rating=0.6157460414323972),
 Rating(user=149, product=9, rating=0.5935115641644458),
 Rating(user=149, product=12, rating=0.5824727319235784),
 Rating(user=149, product=21, rating=0.5797634139589907),
 Rating(user=149, product=10, rating=0.5757244213109232),
 Rating(user=149, product=8, rating=0.5377526253723545),
 Rating(user=149, product=17, rating=0.5316234451108355),
 Rating(user=149, product=14, rating=0.524654945991108),
 Rating(user=149, product=4, rating=0.5106214967170337),
 Rating(user=149, product=5, rating=0.4978389263429476),
 Rating(user=149, product=19, rating=0.49762637207614324),
 Rating(user=149, product=7, rating=0.47406222657664293),
 Rating(user=149, product=1, rating=0.4543212285768474),
 Rating(user=149, product=15, rating=0.3694580203459196),
 Rating(user=149, pr

In [37]:
names = ['User ID', 'Movie ID', 'Ratings']
rec_df = pd.DataFrame(np.array(derived_rec).reshape(len(derived_rec),len(derived_rec[0])), columns = names)
print (rec_df)

    User ID  Movie ID   Ratings
0     149.0      16.0  0.644760
1     149.0      13.0  0.621580
2     149.0       3.0  0.617779
3     149.0      18.0  0.615746
4     149.0       9.0  0.593512
5     149.0      12.0  0.582473
6     149.0      21.0  0.579763
7     149.0      10.0  0.575724
8     149.0       8.0  0.537753
9     149.0      17.0  0.531623
10    149.0      14.0  0.524655
11    149.0       4.0  0.510621
12    149.0       5.0  0.497839
13    149.0      19.0  0.497626
14    149.0       7.0  0.474062
15    149.0       1.0  0.454321
16    149.0      15.0  0.369458
17    149.0       6.0  0.360797
